# Sprint22 深層学習スクラッチ リカレントニューラルネットワーク

## 1.このSprintについて

### Sprintの目的
- スクラッチを通してリカレントニューラルネットワークの基礎を理解する

### どのように学ぶか
- スクラッチでリカレントニューラルネットワークの実装を行います。

## 2.リカレントニューラルネットワークスクラッチ
リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。

クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

### 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$a_t = x_t \cdot W_x + h_{t-1} \cdot W_h + B$$
$$h_t = tanh(a_t)$$

$a_t$  : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)

$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)

$x_t$ : 時刻tの入力 (batch_size, n_features)

$W_x$ : 入力に対する重み (n_features, n_nodes)

$h_{t−1}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)

$W_h$ : 状態に対する重み。 (n_nodes, n_nodes)

$B$ : バイアス項 (n_nodes,)

初期状態 $h_0$ は全て0とすることが多いですが、任意の値を与えることも可能です。

上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 $x$ は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。

分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [5]:
import numpy as np

In [7]:
class ScratchSimpleRNNClassifier:
    def forward(self, X, Wx, Wh, B):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (n_batches, n_sequences, n_features)(1, 3, 2)
            入力
        Returns
        ----------
        A : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)(1, 4)
        H : 時刻tの状態・出力 (batch_size, n_nodes)(1, 4)
        X : 時刻tの入力 (batch_size, n_features)(1, 3)
        Wx : 入力に対する重み (n_features, n_nodes)(2, 4)
        Wh : 状態に対する重み。 (n_nodes, n_nodes)(4, 4)
        B  : バイアス項 (n_nodes,)(4, )
        """
        H = np.zeros((X.shape[0], Wx.shape[1]))
        for i in range(X.shape[1]):
            A_t = (X[:, i, :]@Wx) + (H@Wh) + B
            H = np.tanh(A_t)
        return H       

### 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。

入力$x$、初期状態$h、重み$w_x$と$w_h$、バイアス$b$を次のようにします。

ここで配列$x$の軸はバッチサイズ、系列数、特徴量数の順番です。
```
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)
```

In [8]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)
rnn = ScratchSimpleRNNClassifier()
h = rnn.forward(x, w_x, w_h, b)
print(h)

[[0.79494228 0.81839002 0.83939649 0.85584174]]


フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。
```
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
```